# O PROBLEMA

Você foi contratado(a) para uma consultoria, e seu trabalho envolve analisar os dados de preço do petróleo brent, que pode ser encontrado no site do ipea. Essa base de dados histórica envolve duas colunas: data e preço (em dólares).

Um grande cliente do segmento pediu para que a consultoria desenvolvesse um dashboard interativo e que gere insights relevantes para tomada de decisão. Além disso, solicitaram que fosse desenvovido um modelo de Machine Learning para fazer o forecasting do preço do petróleo.

Seu objetivo é:

- Criar um dashboard interativo com ferramentas de sua escolha.
- Seu dashboard deve fazer parte de um storytelling que traga insights relevantes sobre a variação do preço do petróleo, como situações geopolíticas, crises econômicas, demanda global por energia e etc. Isso pode te ajudar com seu modelo. É obrigatório que você traga pelo menos 4 insights neste desafio.
- Criar um modelo de Machine Learning que faça a previsão do preço do petróleo diariamente (lembre-se de time series). Esse modelo deve estar contemplado em seu storytelling e deve conter o código que você trabalhou, analisando as performances do modelo.
- Criar um plano para fazer o deploy em produção do modelo, com as ferramentas que são necessárias.
- Faça o MVP do seu modelo em produção usando o Streamlit.

Links para as bases de dados: 

http://www.ipeadata.gov.br/ExibeSerie.aspx?module=m&serid=1650971490&oper=view

http://www.ipeadata.gov.br/ExibeSerie.aspx?serid=38590&module=M

Referências: 

https://biblioteca.ibp.org.br/pt-BR/search/23350?exp=brent

https://www.ipea.gov.br/desafios/index.php?option=com_content&view=article&id=2083:catid=28&Itemid=23

https://g1.globo.com/Noticias/Economia_Negocios/0,,MUL940136-9356,00-O+ANO+EM+QUE+O+PETROLEO+ENLOUQUECEU+O+MERCADO.html

https://g1.globo.com/economia/noticia/2015/01/entenda-queda-do-preco-do-petroleo-e-seus-efeitos.html

https://www.maxwell.vrac.puc-rio.br/54305/54305.PDF

## Metodologia

<img src = 'img/CRISP_DM.png'>

Para o desenvolvimento deste projeto, optou por seguir a metodologia CRISP-DM (CRoss Industry Standard Process for Data Mining), largamente utilizada em projetos de dados. A CRISP-DM é composta por 6 etapas, são elas: 

- Análise do negócio (business understanding): essa etapa dedica-se ao entendimento do negócio, isto é, sua produto/serviço, público alvo e quais são as estratégias do setor;

- Análise dos dados (data understanding): uma vez realizada a etapa anterior, inicia-se o entendimento dos dados, que compreende a seleção daqueles que são uteis à resolução da problemática requerida, bem como seu estado (fontes, formatos, etc);

- Preparação dos dados (data preparation): essa etapa dedica-se ao pré-processamento dos dados, em acordo com o que é solicitado pelas soluções que se pretende implementar;

- Modelagem (modeling): a modelagem é a etapa na qual se extrai informações de valor sobre os dados, possibilitando a geração de insights úteis ao negócio, capazes de solucionar a problemática;

- Avaliação (evaluation): momento de avaliação do desempenho do modelo aplicado, isto é, verifica-se se esse respondeu as questões levantadas satisfatoriamente;

- Implementação (deployment): por fim, a implementação é a etapa em que se disponibiliza os resultados e insights obtidos as partes interessadas, comumente por meio de uma ferramenta de data viz ou relatório.

O presente notebook engloba as etapas de análise do negócio, análise dos dados, modelagem e avaliação dos modelos. Um script Python inicia a implementação do projeto, alimentando o modelo selecionado periodicamente com os dados do IPEA, gerando previsões que, assim como os dados da série histórica e acurácia do modelo, são incluídos em base de dados local para serem consumidos em um dashboard interativo desenvolvido no Power BI e exportados para arquivos CSV que, disponibilizados no GitHub, são a base de dados para o MVP desenvolvido no Streamlit. A arquitetura é ilustrada no fluxograma abaixo.

<img src = 'img/arquitetura.png'>

## Análise do negócio

A cotação do petróleo é definida por seu preço, em um determinado momento no mercado onde está sendo negociado. Desse modo, a cotação do petróleo é o resultado da oferta e demanda da commodity no mercado internacional. Nesse caso, a unidade de medida utilizada é de dólares por barril de petróleo. O petróleo Brent é todo o petróleo extraído no Mar do Norte e comercializado na Bolsa de Londres e foi batizado assim porque era extraído de uma base da Shell chamada Brent.

O Brent Crude Oil e o West Texas Intermediate (WTI) são as duas maiores referências comerciais da commodity e a principal diferença entre ambas é originada de seus campos de extração. Enquanto petróleo Brent vem do Mar do Norte, o petróleo WTI encontra-se nos Estados Unidos, sendo negociado, historicamente, com valor menor em relação ao Brent.

A causa principal para essa divergência de valores é que eles são um retrato instantâneo dos preços do petróleo de lugares diferentes: o WTI reflete o mercado dos Estados Unidos e o Brent o do Mar do Norte e Europa. Houve um período em que os dois eram referências igualmente válidas para o preço mundial do petróleo. Contudo, à medida que a produção americana aumentou, o WTI começou a ser afetado pelo surpreendente excesso de oferta nos EUA, que se mostrou maior do que os oleodutos existentes no país podiam suportar.

O petróleo bruto é classificado como leve, médio ou pesado com relação à densidade do líquido, ou como "doce" ou "ácido" com base no conteúdo de enxofre. O petróleo leve e doce - que flutua na água e tem baixo teor de enxofre - é valorizado porque exige um processamento menor para ser convertido em derivados. O WTI e o Brent são tipos leves e doces de petróleo bruto.

Estima-se que mais da metade de todo petróleo do mundo é cotado em termos de petróleo Brent. Sua popularidade vem do fato de ser mais barato, tendo em vista que os poços de petróleo Brent se encontram no mar, o que facilita sua extração em comparação ao WTI, que precisa ser transportado por ferrovias.

O petróleo é negociado de duas formas: mercado à vista e mercado futuro. No mercado à vista, as operações de compra e venda são determinadas com base na oferta e demanda do momento. Uma vez realizada a operação, a liquidação ocorre quase que instantaneamente. Já no mercado futuro, que negocia os chamados “barris de papel”, o petróleo é negociado no “papel”, isto é, com base em um valor monetário de cotação do petróleo, sem que haja a troca física do produto. Os contratos de papel são comprados e vendidos com base nas condições de mercado esperadas em períodos futuros. Os compradores e vendedores no mercado de futuros são investidores, que não possuem qualquer intenção de tomar posse do petróleo real.

A intenção dos investidores é comprar contratos futuros de petróleo, baseados nas cotações WTI e Brent, para fins de lucratividade do capital, adivinhando corretamente se os preços dos barris irão aumentar ou diminuir no futuro. Além disso, alguns produtores utilizam o mercado futuro para se proteger da volatilidade dos preços, travando receitas, que visam cobrir custos e garantir lucros da futura produção de petróleo. As refinarias, por exemplo, compram contratos futuros para travar os custos de compra, tendo em vista uma possível queda do petróleo.

O preço do petróleo é influenciado por fatores como mudanças no valor do dólar, mudanças nas políticas da Organization of Petroleum Exporting Countries (OPEC), mudanças nos níveis de produção e estoque do petróleo, saúde da economia global e implementação ou colapso de acordos internacionais.

## Análise dos dados

Os dados utilizados neste projeto estão disponibilizados em uma tabela no site do IPEA (Instituto de Pesquisa Econômica Aplicada) e serão importados por meio da biblioteca Pandas do Python. A tabela disponibiliza os preços por barril do petróleo bruto tipo Brent, negociados em dias úteis, não incluindo despesa de frete e seguro. Os dados serão então exportados para um arquivo em formato csv, a partir do qual serão trabalhados, evitando problemas de indisponibilidade na fonte. Ambas as variáveis são de interesse e estão exploradas abaixo.

### Importando bibliotecas

In [229]:
import warnings
warnings.filterwarnings(action = 'ignore')

In [230]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

### Importando dados

**Importando dados do IPEA**

In [231]:
df = pd.read_html('http://www.ipeadata.gov.br/ExibeSerie.aspx?module=m&serid=1650971490&oper=view', skiprows=1, thousands='.', decimal=',')[0]
df.head()

,0,1
0,08/01/2024,75.47
1,05/01/2024,78.31
2,04/01/2024,75.79
3,03/01/2024,77.18
4,02/01/2024,76.24


### Tratando dados

In [232]:
def renomeiaColunas(df):
    df.rename(columns={0: 'data', 1: 'preco'}, inplace=True)
    return df

In [233]:
def tipaDados(df):
    df['data'] = df['data'].str.replace('/', '-')
    df['data'] = pd.to_datetime(df['data'], format='%d-%m-%Y')
    return df

In [234]:
def excluiDadosNulos(df):
    if 0 not in df.isna().sum().values: 
        print(f'{df.isna().sum().values.sum()} dados nulos excluídos')
        df = df.dropna()
    else:
        print('Não há dados nulos')
    return df

In [235]:
#excluindo dados nulos
df = excluiDadosNulos(df)
df.head()

Não há dados nulos


,0,1
0,08/01/2024,75.47
1,05/01/2024,78.31
2,04/01/2024,75.79
3,03/01/2024,77.18
4,02/01/2024,76.24


In [236]:
#renomeando colunas
df = renomeiaColunas(df)
df.head()

,data,preco
0,08/01/2024,75.47
1,05/01/2024,78.31
2,04/01/2024,75.79
3,03/01/2024,77.18
4,02/01/2024,76.24


In [237]:
#alterando tipo da coluna de data
df = tipaDados(df)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11082 entries, 0 to 11081
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   data    11082 non-null  datetime64[ns]
 1   preco   11082 non-null  float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 173.3 KB


### EDA

In [238]:
df_eda = df.sort_values('data', ascending=False).set_index('data')
df_eda.head()

,preco
data,
2024-01-08,75.47
2024-01-05,78.31
2024-01-04,75.79
2024-01-03,77.18
2024-01-02,76.24


In [239]:
#configurando template do plotly
template = 'ggplot2'

**Estatísticas descritivas**

In [240]:
df_eda.describe()

,preco
count,11082.000000
mean,52.771962
std,33.235140
min,9.100000
25%,20.400000
50%,47.845000
75%,75.667500
max,143.950000


**Distribuição das variáveis**

In [241]:
fig = px.histogram(
    data_frame=df_eda, 
    x='preco', 
    nbins=20,
    template=template,
    color_discrete_sequence=['#00008B']
)
fig.update_layout(
    title='Histograma do Preço do Petróleo Brent',
    xaxis_title='Preço',
    yaxis_title='Frequência',
    width=1600, 
    height=600,
    margin=dict(l=100, r=20, t=80, b=80)
)
fig.show()

In [242]:
df_log = np.log(df_eda)

fig = px.histogram(
    data_frame=df_log, 
    x='preco', 
    nbins=20,
    template=template,
    color_discrete_sequence=['#00008B']
)
fig.update_layout(
    title='Histograma do Preço do Petróleo Brent',
    xaxis_title='Preço',
    yaxis_title='Frequência',
    width=1600, 
    height=600,
    margin=dict(l=100, r=20, t=80, b=80)
)
fig.show()

In [243]:
fig = px.box(
    data_frame=df_eda, 
    x = 'preco',
    template = template,
    color_discrete_sequence = ['#00008B']
)
fig.update_layout(
    title = 'Boxplot do Preço do Petróleo Brent',
    xaxis_title = 'Preço (US$)',
    yaxis_title = 'Volume',
    width = 1600, 
    height = 600,
    margin = dict(l = 80, r = 20, t = 80, b = 80),
    template = template
)
fig.show()

#### Analisando a série

In [244]:
from statsmodels.tsa.seasonal import seasonal_decompose #lib para decompor série temporal

In [245]:
resultados = seasonal_decompose(df_eda, period=5)

In [246]:
media_movel = resultados.trend.to_frame()
sazonalidade = resultados.seasonal.to_frame()
residuo = resultados.resid.to_frame()

fig = make_subplots(rows = 4, cols = 1)
fig.add_trace(go.Scatter(x=df_eda.index, y=df.preco, name='Série', marker=dict(color='#00008B')), row=1, col=1)
fig.add_trace(go.Scatter(x=media_movel.index, y=media_movel.trend, name='Media móvel', marker=dict(color='#B22222')), row=2, col=1)
fig.add_trace(go.Scatter(x=sazonalidade.index, y=sazonalidade.seasonal, name='Sazonalidade', marker = dict(color='#008000')), row=3, col=1)
fig.add_trace(go.Scatter(x=residuo.index, y=residuo.resid, name='Resíduo', marker=dict(color='gold')), row=4, col=1)
fig.update_layout(
    title={
        'text': 'Decomposição da Série de Preços do Petróleo Brent',
        'x': 0.5,
        'xanchor': 'center'
    },
    width=1600, 
    height=1200,
    template=template
)
fig.show()

**Analisando sazonalidade**

In [247]:
df_eda = df_eda.loc[df_eda.index >= '2023-01-01']
resultados_teste = seasonal_decompose(df_eda, period=5)
sazonalidade_teste = resultados_teste.seasonal.to_frame()

fig = px.line(
    sazonalidade_teste,
    template=template,
    color_discrete_sequence=['#008000']
)
fig.update_layout(
    title='Sazonalidade da Série de Preços do Petróleo Brent',
    xaxis_title='Período',
    yaxis_title='Sazonalidade',
    showlegend=False,
    width=1600, 
    height=600,
    margin=dict(l=100, r=20, t=80, b=80)
)
fig.show()

**Analisando oscilações**

Verifica-se três grandes oscilações negativas na série, com inícios em 2008, 2014 e 2020. 

- 2008: a chamada terceira crise do petróleo está relacionada a especulação imobiliária nos Estados Unidos, o subprime, que provocou um aumento abusivo nos valores dos imóveis, fazendo com que as hipotecas acabassem não tendo a liquidez esperada, elevando juros e inflação. A consequência desse desastre econômico que colocou em xeque o capitalismo foi desemprego em massa, retração financeira internacional, principalmente na Europa. Com isso, houve o aumento da dívida pública externa por conta da necessidade de empréstimos junto ao Fundo Monetário Internacional (FMI). Esse cenário causou uma queda drástica no valor e nas demandas por commodities, como o petróleo. Ademais, no primeiro semestre desse ano, uma soma de fatores levou os preços: tensões geopolíticas, do Irã à Nigéria passando pelo Paquistão, o equilíbrio tenso entre uma oferta limitada e uma demanda puxada pelos países emergentes, a conscientização de que as reservas são limitadas e de acesso cada vez mais difícil e uma febre dos fundos de investimento por matérias-primas. Esses fundos usaram o petróleo de investimento contra a inflação da commodity. Temendo a alta dos preços, eles acabaram por alimentá-la fazendo subirem os preços do petróleo, mas depois da falência do banco americano Lehman Brothers em setembro, esta lógica se inverte. Temendo a deflação, os investidores abandonam o petróleo, porque precisam urgentemente de liquidez. Ao mesmo tempo, o petróleo caro do primeiro semestre derruba o consumo de combustível dos países industrializados, o que derrubou ainda mais a demanda.

- 2014: é o pior tombo de preços desde 2008, causada pelo aumento de produção, em especial nas áreas de xisto dos EUA, uma demanda menor que a esperada na Europa e na Ásia e a recusa dos países da Organização dos Países Exportadores de Petróleo (OPEP) em reduzir seu teto de produção, independentemente do preço no mercado internacional. Com a superprodução, oriunda sobretudo do petróleo de xisto dos EUA, os membros da OPEP, visando não perder mercado, mantiveram suas produções afim de tornar insustentável a produção norte-americana. Alguns países sofrem particularmente com a redução dos preços do petróleo, sobretudo Venezuela, Rússia e Irã, em razão do grande peso das exportações da commodity em suas economias.

- 2020: o setor petrolífero viveu um momento de instabilidade em razão da pandemia do novo coronavírus. Nesta perspectiva, as medidas de contenção do contágio do vírus, como o isolamento social, afetaram todo o mercado mundial em uma crise generalizada na demanda de produtos e consequente distorção de preços (SILVEIRA, 2020). No caso do petróleo, presente em diversos mercados, houve drástica redução do consumo - principalmente, o West Texas Intermediate (WTI). Essa diminuição causou uma distorção nos preços da commodity, que caíram excessivamente, gerando um desequilíbrio entre oferta e demanda. Essa redução expressiva do consumo atingiu os grandes produtores membros da OPEP, cujos membros vinham, desde 2016, discutindo a possibilidade de redução do volume de produção para alavancar os preços da commodity e drenar a alta quantidade em estoque. Diante do cenário gerado pela COVID-19, a necessidade da medida se tornou ainda mais evidente e a Arábia Saudita propôs um acordo com a Rússia que previa a redução na produção em 1,5 milhão de barris, visando estabilizar o preço do petróleo e manter a competitividade dos dois gigantes do mercado petrolífero. Entretanto, a Rússia, não concordou em reduzir sua produção, pelo receio em perder espaço no mercado competitivo para o petróleo WTI. Em resposta à Rússia, a Arábia Saudita decidiu rebaixar seus preços a níveis de 20 anos atrás e, ao mesmo tempo, aumentar sua produção para 12,3 milhões de barris por dia a partir de abril, dando início ao que é conhecido como “a guerra dos preços”.

## Preparação dos dados

In [248]:
from statsforecast import StatsForecast
from statsforecast.models import AutoARIMA
from prophet import Prophet

**Tratando DataFrame para as bibliotecas StatsForecast e Prophet**

A biblioteca Statsforecast reúne os modelos estatísticos de previsão de séries temporais mais populares numa interface simples, não limitado a prever uma série por vez.

In [249]:
df_preparado = df[['data', 'preco']].rename(columns={'data': 'ds', 'preco': 'y'})
df_preparado['unique_id'] = 'Preco'
df_preparado.dropna(inplace=True)
df_preparado.head()

,ds,y,unique_id
0,2024-01-08,75.47,Preco
1,2024-01-05,78.31,Preco
2,2024-01-04,75.79,Preco
3,2024-01-03,77.18,Preco
4,2024-01-02,76.24,Preco


**Separando base de treino e de teste para o Statsforecast**

In [250]:
treino_arima = df_preparado.loc[(df_preparado['ds'] >= '2000-01-01') & (df_preparado['ds'] < '2024-01-02')] #dados de treino
teste_arima = df_preparado.loc[df_preparado['ds'] >= '2024-01-02'] #dados de validação (5 dias), quanto maior o período de predição, maior a largura de banda (maior o erro)
h = teste_arima['ds'].nunique() #datas distintas no intervalo (5 dias)
h

5

In [251]:
teste_arima.shape, treino_arima.shape

((5, 3), (7877, 3))

**Separando base de treino e de teste para o Prophet**

In [252]:
treino_prophet = treino_arima.reset_index(drop=True).drop('unique_id', axis=1)
teste_prophet = teste_arima.drop('unique_id', axis=1).set_index('ds', drop=True)
h

5

In [253]:
teste_prophet.shape, treino_prophet.shape

((5, 1), (7877, 2))

## Modelagem

### ARIMA

Se combinarmos a diferenciação com a autorregressão e um modelo de média móvel, obtemos um modelo ARIMA não sazonal. ARIMA é um acrônimo para Média Móvel Integrada AutoRegressiva (neste contexto, “integração” é o inverso de diferenciação). O modelo completo pode ser escrito como

### $$y'_{t} = c + \phi_{1}y'_{t-1} + \cdots + \phi_{p}y'_{t-p} + \theta_{1}\varepsilon_{t-1} + \cdots + \theta_{q}\varepsilon_{t-q} + \varepsilon_{t},$$

onde $y't$ é a série diferenciada (pode ter sido diferenciada mais de uma vez).

Os “preditores” no lado direito incluem ambos os valores defasados ​​de $yt$ e erros atrasados. Chamamos isso de modelo ARIMA($p, d, q$) modelo, onde:

- $p$ = ordem da parte autoregressiva;

- $d$ = grau de primeira diferenciação envolvido;

- $q$ = ordem da parte da média móvel.

As mesmas condições de estacionariedade e invertibilidade usadas para modelos autoregressivos e de média móvel também se aplicam a um modelo ARIMA.



In [254]:
modelo_arima = StatsForecast(models=[AutoARIMA(season_length=5)], freq='B', n_jobs=-1)
modelo_arima.fit(treino_arima)
forecast_arima = modelo_arima.predict(h=h, level=[90])
forecast_arima.ds = teste_arima.ds.to_list()
forecast_arima = forecast_arima.reset_index().merge(teste_arima, on =['ds', 'unique_id'], how='left')
forecast_arima

,unique_id,ds,AutoARIMA,AutoARIMA-lo-90,AutoARIMA-hi-90,y
0,Preco,2024-01-08,78.319946,75.615746,81.024139,75.47
1,Preco,2024-01-05,78.584236,75.070709,82.097771,78.31
2,Preco,2024-01-04,78.466019,74.553566,82.378479,75.79
3,Preco,2024-01-03,78.414330,74.105713,82.722939,77.18
4,Preco,2024-01-02,78.436493,73.725357,83.147621,76.24


In [255]:
#gerando linha contínua para banda de confiança (é preciso uma linha contínua para aplicar o preenchimento)
banda_sup_arima = forecast_arima['AutoARIMA-hi-90'].tolist() #limite superior
banda_inf_arima = forecast_arima['AutoARIMA-lo-90'].tolist() #limite inferior
banda_arima = banda_sup_arima + banda_inf_arima[::-1] #concatenando banda superior e banda inferior invertida 
banda_arima_index = forecast_arima.ds.tolist()
banda_arima_index = banda_arima_index + banda_arima_index[::-1] #concatenando datas em ordem crescente com ordem decrescente 

In [256]:
#concatenando treino e teste
treino_px = treino_arima.drop(columns = 'unique_id').set_index('ds') 
treino_px = treino_px[(treino_px.index >= '2023-09-01') & (treino_px.index < '2024-01-08')]
teste_px = teste_arima.drop(columns = 'unique_id').set_index('ds')
base_arima_px = pd.concat([teste_px, treino_px]).rename(columns = {'y': 'Preço'})
base_arima_px.head()

,Preço
ds,
2024-01-08,75.47
2024-01-05,78.31
2024-01-04,75.79
2024-01-03,77.18
2024-01-02,76.24


In [257]:
fig = px.line(
    base_arima_px,
    template=template,
    color_discrete_sequence=['#00008B']
)
fig.add_scatter(
    x=banda_arima_index, 
    y=banda_arima,
    name='Banda de confiança',
    mode='lines',
    line_color='#B22222',
    line_width=0,
    opacity=0.5,
    fill='toself'
)
fig.add_scatter(
    x=forecast_arima.ds, 
    y=forecast_arima['AutoARIMA'], 
    name='Preço previsto',
    marker=dict(color='#B22222')
)
fig.update_layout(
    title='ARIMA',
    xaxis_title='Período',
    yaxis_title='Preço do Petróleo Brent',
    legend_title='Legenda',
    width=1600, 
    height=600,
    margin=dict(l=100, r=20, t=80, b=80)
)
fig.show()

### Prophet

O Prophet é um modelo produzido pelo Facebook com o objetivo de prever valores futuros com base em dados históricos. Ele funciona identificando padrões nos dados, como tendências que aumentam ou diminuem ao longo do tempo e combinando tendências, sazonalidades e efeitos em uma equação matemática.

### $$y_t = g(t) + s(t) + h(t) + \varepsilon_t,$$ 

onde:

- $g(t)$ = tendência linear ao longo do tempo;

- $s(t)$ = padrões sazonais;

- $h(t)$ = efeitos de feriados;

- $ε$ = termo de erro em forma de ruído.

In [258]:
modelo_prophet = Prophet()
modelo_prophet.fit(treino_prophet)

21:11:31 - cmdstanpy - INFO - Chain [1] start processing
21:11:37 - cmdstanpy - INFO - Chain [1] done processing


In [259]:
future = modelo_prophet.make_future_dataframe(periods=10, freq= 'D', include_history=False)
future = future.iloc[[3, 4, 5, 6, 9]].reset_index(drop=True) #excluindo final de semana
future

,ds
0,2024-01-02
1,2024-01-03
2,2024-01-04
3,2024-01-05
4,2024-01-08


In [260]:
forecast_prophet = modelo_prophet.predict(future) #realizando previsão
forecast_prophet = forecast_prophet[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
forecast_prophet

,ds,yhat,yhat_lower,yhat_upper
0,2024-01-02,84.187589,70.157784,97.914763
1,2024-01-03,84.159563,70.248516,97.843862
2,2024-01-04,84.292699,69.832986,97.710969
3,2024-01-05,84.395004,71.051300,97.858945
4,2024-01-08,84.568084,71.893197,97.905157


In [261]:
#gerando linha contínua para banda de confiança (é preciso uma linha contínua para aplicar o preenchimento)
banda_sup_prophet = forecast_prophet['yhat_upper'].tolist() #limite superior
banda_inf_prophet = forecast_prophet['yhat_lower'].tolist() #limite inferior
banda_prophet = banda_sup_prophet + banda_inf_prophet[::-1] #concatenando banda superior e banda inferior invertida 
banda_prophet_index = forecast_prophet.ds.tolist()
banda_prophet_index = banda_prophet_index + banda_prophet_index[::-1] #concatenando datas em ordem crescente com ordem decrescente

In [262]:
#concatenando treino e teste
treino_px = treino_prophet.set_index('ds') 
treino_px = treino_px[(treino_px.index >= '2023-09-01') & (treino_px.index < '2024-01-08')]
teste_px = teste_prophet
base_prophet_px = pd.concat([teste_px, treino_px]).rename(columns = {'y': 'Preço'})
base_prophet_px.head()

,Preço
ds,
2024-01-08,75.47
2024-01-05,78.31
2024-01-04,75.79
2024-01-03,77.18
2024-01-02,76.24


In [263]:
forecast_prophet = pd.merge(forecast_prophet, teste_prophet, on='ds')
forecast_prophet = forecast_prophet.set_index('ds')
forecast_prophet

,yhat,yhat_lower,yhat_upper,y
ds,,,,
2024-01-02,84.187589,70.157784,97.914763,76.24
2024-01-03,84.159563,70.248516,97.843862,77.18
2024-01-04,84.292699,69.832986,97.710969,75.79
2024-01-05,84.395004,71.051300,97.858945,78.31
2024-01-08,84.568084,71.893197,97.905157,75.47


In [264]:
fig = px.line(
    base_prophet_px, 
    template=template,
    color_discrete_sequence=['#00008B']
)
fig.add_scatter(
    x=banda_prophet_index, 
    y=banda_prophet,
    name='Banda de confiança',
    mode='lines',
    line_color='#B22222',
    line_width=0,
    opacity=0.5,
    fill='toself'
)
fig.add_scatter(
    x=forecast_prophet.index, 
    y=forecast_prophet.yhat, 
    name='Preço previsto',
    marker=dict(color='#B22222')
)
fig.update_layout(
    title='Prophet',
    xaxis_title='Período',
    yaxis_title='Preço do Petróleo Brent',
    legend_title='Legenda',
    width=1600, 
    height=600,
    margin=dict(l=100, r=20, t=80, b=100)
)
fig.show()

## Avaliação

### Função de validação

Erro absoluto percentual médio (MAPE) é uma métrica para avaliar modelos de regressão em machine learning. Um valor de MAPE baixo significa que as previsões estão próximas dos valores observados, enquanto um valor alto indica que as previsões estão longe dos valores observados (o modelo precisa ser melhorado).

$$\text{MAPE}(y, \hat{y}) = \frac{1}{N} \sum_{i=0}^{n - 1} \text{|} \frac{y_i - \hat{y}_i}{y_i} \text{|}$$

O WMAPE (Weighted Mean Absolute Percentage Error) é uma variação do MAPE que faz as somas dos erros percentuais e dos valores observados antes da divisão. O WAPE é outra maneira de escapar do problema de valores zero nos dados observados. Ele é outra alternativa recomendada quando a maioria de seus produtos não vendem em todos os períodos ou são vendidos em pouca quantidade. Quando um produto tem baixo volume de vendas, qualquer erro na previsão tem um impacto significativo na porcentagem de erro calculada pelo MAPE. Mesmo que o erro absoluto seja pequeno, ele será dividido por um valor muito baixo, resultando em uma porcentagem de erro elevada. Isso pode dar a impressão que a previsão é muito pior do que realmente é. Já no caso do WAPE, como a divisão é feita pela soma de todos os valores reais absolutos de uma vez, esse efeito é menor.

$$\text{WMAPE}(y, \hat{y}) = \frac{\sum_{i=1}^{n} \text{|} \hat{y}_i - {y_i} \text{|}}{\sum_{i=1}^{n} \text{|} {y}_i \text{|}}

In [265]:
#função para validação dos modelos
def wmape(y_true, y_pred):
    return np.abs(y_true - y_pred).sum() / np.abs(y_true).sum()

In [266]:
wmape_arima = wmape(forecast_arima['y'].values, forecast_arima['AutoARIMA'].values)
print(f'WAPE: {wmape_arima}')

WAPE: 0.024102518498055572


In [267]:
wmape_prophet = wmape(forecast_prophet['y'].values, forecast_prophet['yhat'].values)
print(f'WAPE: {wmape_prophet:.2%}')

WAPE: 10.08%


**Exportando modelo**

In [268]:
import joblib
from statsforecast import StatsForecast
from statsforecast.models import AutoARIMA

df_preparado = df[['data', 'preco']].rename(columns={'data': 'ds', 'preco': 'y'})
df_preparado['unique_id'] = 'Preco'
df_preparado.dropna(inplace=True)

treino = df_preparado.loc[df_preparado['ds'] >= '2000-01-01'] #novos dados de treino
h = 5

modelo = StatsForecast(models=[AutoARIMA(season_length=5)], freq='B', n_jobs=-1)
modelo.fit(treino)
joblib.dump(modelo, 'modelos/auto_arima.joblib')

modelo = joblib.load('modelos/auto_arima.joblib')
forecast = modelo.predict(h=h, level=[90])
forecast = forecast[['ds', 'AutoARIMA']].reset_index(drop=True).rename(columns={'ds': 'data', 'AutoARIMA': 'preco_previsto_brent'})
forecast['preco_previsto_brent'] = [int(n * 100) / 100 for n in forecast['preco_previsto_brent']]
forecast

,data,preco_previsto_brent
0,2024-01-09,75.44
1,2024-01-10,75.98
2,2024-01-11,75.99
3,2024-01-12,75.89
4,2024-01-15,75.89


## Implementação

Os dados do IPEA serão obtidos via Python, semanalmente, em um arquivo web scraping que realizará a inserção dos dados no banco local para serem consumidos em um dashboard Power BI e que também realizará a exportação de arquivos CSV que, disponibilizados no GitHub, serão a base de dados para o MVP desenvolvido no Streamlit.